In [1]:
# -*- coding: utf-8 -*-
"""
Exemple simple de MNIST avec PyTorch
Exemple avec nn.Sequential, nn.LogSoftmax, nn.NLLLoss
"""
import torch
torch.manual_seed(0) # Pour résultats reproductibles

# Fonction J d'entropie croisée
import torch.nn as nn
fonction_cout = nn.NLLLoss()

modele = nn.Sequential(
    nn.Linear(784, 30),
    nn.ReLU(),
    nn.Linear(30, 10),
    nn.LogSoftmax(dim=1)
)
    
from torch import optim
optimiseur = optim.SGD(modele.parameters(), lr=0.05)

# Chargement des données de MNIST
import pickle, gzip
fichier_donnees = gzip.open(r"mnist.pkl.gz", 'rb')
((donnees_ent_X, donnees_ent_Y),(donnees_valid_X, donnees_valid_Y),(donnees_test_X,donnees_test_Y)) = pickle.load(fichier_donnees, encoding="latin-1")
fichier_donnees.close()

# Conversion des données en type toch.Tensor
import torch
donnees_ent_X, donnees_ent_Y, donnees_test_X,donnees_test_Y = map(torch.tensor, (donnees_ent_X, donnees_ent_Y, donnees_test_X,donnees_test_Y))

# Création des objets DataLoader pour itérer par lot
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
ds_ent = TensorDataset(donnees_ent_X, donnees_ent_Y)
dl_ent = DataLoader(ds_ent, batch_size=100, shuffle=True)

nb_epochs = 10
# Boucle d'apprentissage
for epoch in range(nb_epochs):
    cout_total_ent = 0 
    
    # Boucle d'apprentissage par mini-lot pour une epoch
    for lot_X, lot_Y in dl_ent:
        optimiseur.zero_grad() # Remettre les dérivées à zéro
        lot_Y_predictions = modele(lot_X) # Appel de la méthode forward
        cout = fonction_cout(lot_Y_predictions, lot_Y)
        cout_total_ent +=cout
        cout.backward() # Calcul des gradiants par rétropropagation
        optimiseur.step() # Mise à jour des paramètres
        
    cout_moyen_ent = cout_total_ent/len(dl_ent)
    print(f'-------- > epoch {epoch+1}:  coût moyen entraînement = {cout_moyen_ent}')

-------- > epoch 1:  coût moyen entraînement = 0.7773018479347229
-------- > epoch 2:  coût moyen entraînement = 0.3552558422088623
-------- > epoch 3:  coût moyen entraînement = 0.3122105598449707
-------- > epoch 4:  coût moyen entraînement = 0.2863697111606598
-------- > epoch 5:  coût moyen entraînement = 0.2668796181678772
-------- > epoch 6:  coût moyen entraînement = 0.2510523796081543
-------- > epoch 7:  coût moyen entraînement = 0.23606045544147491
-------- > epoch 8:  coût moyen entraînement = 0.22344647347927094
-------- > epoch 9:  coût moyen entraînement = 0.21195107698440552
-------- > epoch 10:  coût moyen entraînement = 0.20151516795158386


In [2]:
X = torch.randn(size=(1,784), dtype=torch.float32)
for une_couche in modele:
    X = une_couche(X)
    print('Type de couche:',une_couche.__class__.__name__,'  Forme de sortie:',X.shape)

Type de couche: Linear   Forme de sortie: torch.Size([1, 30])
Type de couche: ReLU   Forme de sortie: torch.Size([1, 30])
Type de couche: Linear   Forme de sortie: torch.Size([1, 10])
Type de couche: LogSoftmax   Forme de sortie: torch.Size([1, 10])


In [3]:
print(modele)

Sequential(
  (0): Linear(in_features=784, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=10, bias=True)
  (3): LogSoftmax()
)
